In [1]:
%use multik

In [2]:
data class Star(
    var id : Int? = null,
    var hip : Int? = null,
    var hd : Int? = null,
    var hr : Int? = null,
    var gl : String? = null,
    var bf : String? = null,
    var proper : String? = null,
    var ra : Double? = null,
    var dec : Double? = null,
    var dist : Double? = null,
    var pmra : Double? = null,
    var pmdec : Double? = null,
    var rv : Double? = null,
    var mag : Double? = null,
    var absmag : Double? = null,
    var spect : String? = null,
    var ci : Double? = null,
    var x : Double? = null,
    var y : Double? = null,
    var z : Double? = null,
    var vx : Double? = null,
    var vy : Double? = null,
    var vz : Double? = null,
    var rarad : Double? = null,
    var decrad : Double? = null,
    var pmrarad : Double? = null,
    var pmdecrad : Double? = null,
    var bayer : String? = null,
    var flam : Int? = null,
    var con : String? = null,
    var comp : Int? = null,
    var comp_primary : Int? = null,
    var base : String? = null,
    var lum : Double? = null,
    var variable : String? = null,
    var var_min : Double? = null,
    var var_max : Double? = null,
)

In [3]:
var typeList: MutableList<String> = mutableListOf("Int?", "Int?", "Int?", "Int?", "String?", "String?", "String?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "String?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "Double?", "String?", "Int?", "String?", "Int?", "Int?", "String?", "Double?", "String?", "Double?", "Double?")

In [5]:
fun changeType(data: String, type: String): Any?{
    when(type){
        "Int?" -> return data.toIntOrNull()
        "Double?" -> return data.toDoubleOrNull()
        "String?" -> return data
        else -> throw IllegalAccessException()
    }
}

In [6]:
import java.io.BufferedReader
import java.io.File
import java.io.FileReader
import kotlin.reflect.KMutableProperty
import kotlin.reflect.full.memberProperties
import kotlin.streams.toList

fun readCsv(fileName: String, typeList: MutableList<String>): MutableList<Star> {
    val file = File(fileName)
    val reader = BufferedReader(FileReader(file))

    val header = reader.readLine()
    var properList = header.split(",").toMutableList()

    for ((i, item) in properList.withIndex()) {
        if (item == "var") {
            properList[i] = "variable"
        }
    }

    val data = reader.lines().toList()
    var i = 0

    var starList: MutableList<Star> = mutableListOf()
    for (line in data) {
        i++
        var star = Star()
        val stringData = line.split(',')

        for ((i, propertyName) in properList.withIndex()) {
            val property = star::class.memberProperties.find { it.name == propertyName }

            val type = typeList[i]
            val value = changeType(stringData[i], type)

            if (property != null && property is KMutableProperty<*>) {
                property.setter.call(star, value)
            }
        }
        starList.add(star)
    }
    return starList
}
var starList = readCsv("./hyg_v37.csv", typeList)

In [7]:
var new_starList = starList.subList(1, starList.size)

In [8]:
println("ra:${new_starList[1000].rarad} dec:${new_starList[1000].decrad}")

ra:0.054757002 dec:0.429985749


In [9]:
var stars = mutableListOf<List<Double>>()

for(star in new_starList){
    val ra = star.rarad?.toDouble() as Double
    val dec = star.decrad?.toDouble() as Double
    val lis = listOf(cos(ra) * cos(dec), cos(dec) * sin(ra), sin(dec))
    stars.add(lis)
}

var starList = stars.toList()

In [10]:
var starArray =  mk.ndarray(starList)
println(starArray)

[[0.9426482486234852, 6.971930150839365E-5, -0.33378776865891185],
[0.7786892523555762, 6.823365820832647E-5, 0.6274097892207944],
[0.6171245124403928, 9.192316530588837E-5, -0.7868655080096847],
[0.7593712128157781, 1.321556516141891E-4, -0.6506576240094155],
[0.9976287483259759, 3.254504523389332E-4, 0.06882422971272313],
[0.9394767860279152, 3.614383453833962E-4, 0.34261237262682376],
[0.8996608704293885, 4.2926242097032834E-4, 0.43658920503375825],
[0.8029634190013711, 4.950775972443158E-4, 0.5960281055821119],
[0.6311228679984664, 4.004639226849576E-4, -0.7756827735086276],
[0.6827637574002127, 4.447680780496505E-4, 0.7306390721567663],
[0.8094246364394597, 5.778402518710655E-4, -0.5872234872903942],
[0.9232454832828836, 6.719418743574573E-4, -0.3842100025921663],
[0.9999798544220785, 8.451041757484965E-4, -0.006290989503318646],
[0.6321482492708922, 5.554327771180033E-4, 0.7748472639418533],
[0.7638814205851765, 6.801533216972399E-4, -0.6453562680227345],
[0.5747999026342704, 5.2

In [11]:
fun makeLocation(starList: List<Star>): NDArray<*, *>{
    var stars = mutableListOf<List<Double>>()

    for(star in new_starList){
        val ra = star.rarad?.toDouble() as Double
        val dec = star.decrad?.toDouble() as Double
        val lis = listOf(cos(ra) * cos(dec), cos(dec) * sin(ra), sin(dec), star.id?.toDouble() as Double, star.mag as Double)
        stars.add(lis)
    }

    var starList = stars.toList()
    return mk.ndarray(starList)
}

In [12]:
var starArray = makeLocation(new_starList)

In [13]:
fun getXAxis(theta: Double, phi: Double): List<Double>{
    return listOf(cos(theta) * cos(phi), sin(theta) * cos(phi), sin(phi), 1.0, 1.0)
}

fun getZAxis(theta: Double, phi: Double): List<Double>{
    return getXAxis(theta, phi + PI * 0.5)
}

fun crossProduct(arr1: List<Double>, arr2: List<Double>): List<Double>{
    return listOf(arr1[1] * arr2[2] - arr1[2] * arr2[1], arr1[2] * arr2[0] - arr1[0] * arr2[2], arr1[0] * arr2[1] - arr2[0] * arr1[1], 1.0, 1.0)
}

fun getYAxis(theta: Double, phi: Double): List<Double>{
    return crossProduct(getZAxis(theta, phi), getXAxis(theta, phi))
}

fun getTransformMatrix(theta: Double, phi: Double): NDArray<*, *>{
    val new_theta = theta * PI / 180
    val new_phi = phi * PI / 180
    return mk.ndarray(listOf(getXAxis(new_theta, new_phi), getYAxis(new_theta, new_phi), getZAxis(new_theta, new_phi)))
}

In [14]:
fun changeCoords(theta: Double, phi: Double, starArray: NDArray<*, *>): NDArray<*, *> {
    val trans = getTransformMatrix(theta, phi)
    return mk.linalg.dot(trans.asDNArray() as MultiArray<Double, D2>, starArray.transpose().asDNArray() as MultiArray < Double, D2 >)
}

val b = changeCoords(60.0, -30.0, starArray)